# Exercice 03 - Decouverte de Spark

## Objectifs
- Comprendre ce qu'est Apache Spark
- Creer une SparkSession
- Comprendre la difference entre Spark et Pandas
- Creer votre premier DataFrame Spark

---

## 1. Qu'est-ce que Apache Spark ?

**Apache Spark** est un moteur de traitement de donnees distribue.

```
+------------------------------------------------------------------+
|                    TRAITEMENT PANDAS                             |
|                                                                  |
|   +----------+          +------------------+                     |
|   | Donnees  | -------> | 1 seul processeur| -----> Resultat     |
|   +----------+          +------------------+                     |
|                                                                  |
|   Limite : tout doit tenir en memoire RAM                        |
+------------------------------------------------------------------+

+------------------------------------------------------------------+
|                    TRAITEMENT SPARK                              |
|                                                                  |
|   +----------+     +-------+  +-------+  +-------+               |
|   | Donnees  | --> | CPU 1 |  | CPU 2 |  | CPU 3 | --> Resultat  |
|   | (huge)   |     +-------+  +-------+  +-------+               |
|   +----------+         |          |          |                   |
|                        +----------+----------+                   |
|                               |                                  |
|                        Coordination                              |
|                                                                  |
|   Avantage : distribue le travail sur plusieurs machines         |
+------------------------------------------------------------------+
```

### Caracteristiques principales
- **Distribue** : repartit le travail sur plusieurs machines
- **En memoire** : traite les donnees en RAM (tres rapide)
- **Polyvalent** : SQL, Machine Learning, Streaming, Graphes
- **Compatible** : lit CSV, JSON, Parquet, bases de donnees...

## 2. Spark vs Pandas

| Critere | Pandas | Spark |
|---------|--------|-------|
| Execution | 1 machine | Cluster de machines |
| Taille donnees | < 10 Go | Petaoctets |
| Vitesse petits fichiers | Tres rapide | Plus lent (overhead) |
| Vitesse gros fichiers | Lent/impossible | Tres rapide |
| API | DataFrame | DataFrame |
| Langage | Python | Python, Scala, Java, R |

### Quand utiliser quoi ?
- **Pandas** : fichiers < 1 Go, exploration rapide, prototypage
- **Spark** : fichiers > 1 Go, production, pipelines Data Lake

## 3. La SparkSession

La **SparkSession** est le point d'entree pour utiliser Spark.

C'est comme ouvrir une connexion a Spark avant de pouvoir travailler.

```
+------------------------------------------------------------------+
|                                                                  |
|   Votre code Python                                              |
|        |                                                         |
|        v                                                         |
|   +--------------+                                               |
|   | SparkSession | <-- Point d'entree unique                     |
|   +--------------+                                               |
|        |                                                         |
|        v                                                         |
|   +------------------+                                           |
|   | Moteur Spark     |                                           |
|   | (distribue)      |                                           |
|   +------------------+                                           |
|                                                                  |
+------------------------------------------------------------------+
```

## 4. Creer une SparkSession

In [1]:
# Importer SparkSession depuis le module pyspark.sql
from pyspark.sql import SparkSession

# Creer une SparkSession
# - appName : nom de votre application (visible dans l'interface Spark)
# - getOrCreate : reutilise une session existante ou en cree une nouvelle

spark = SparkSession.builder \
    .appName("Mon premier Spark") \
    .getOrCreate()

print("SparkSession creee avec succes !")
print("Version Spark :", spark.version)

SparkSession creee avec succes !
Version Spark : 4.0.1


In [2]:
# Afficher les informations de la session
print("Application :", spark.sparkContext.appName)
print("Master :", spark.sparkContext.master)

Application : Mon premier Spark
Master : local[*]


## 5. Creer un DataFrame Spark

Un **DataFrame Spark** est similaire a un DataFrame Pandas, mais distribue.

In [3]:
# Creer un DataFrame a partir d'une liste de tuples
donnees = [
    ("Alice", 30, "Paris"),
    ("Bob", 25, "Lyon"),
    ("Charlie", 35, "Marseille"),
    ("Diana", 28, "Bordeaux")
]

# Definir les noms des colonnes
colonnes = ["nom", "age", "ville"]

# Creer le DataFrame
df = spark.createDataFrame(donnees, colonnes)

# Afficher le DataFrame
df.show()

+-------+---+---------+
|    nom|age|    ville|
+-------+---+---------+
|  Alice| 30|    Paris|
|    Bob| 25|     Lyon|
|Charlie| 35|Marseille|
|  Diana| 28| Bordeaux|
+-------+---+---------+



In [4]:
# Afficher le schema (structure) du DataFrame
df.printSchema()

root
 |-- nom: string (nullable = true)
 |-- age: long (nullable = true)
 |-- ville: string (nullable = true)



In [5]:
# Compter le nombre de lignes
print("Nombre de lignes :", df.count())

Nombre de lignes : 4


## 6. Operations de base sur un DataFrame

In [6]:
# Selectionner certaines colonnes
df.select("nom", "ville").show()

+-------+---------+
|    nom|    ville|
+-------+---------+
|  Alice|    Paris|
|    Bob|     Lyon|
|Charlie|Marseille|
|  Diana| Bordeaux|
+-------+---------+



In [7]:
# Filtrer les lignes
df.filter(df.age > 27).show()

+-------+---+---------+
|    nom|age|    ville|
+-------+---+---------+
|  Alice| 30|    Paris|
|Charlie| 35|Marseille|
|  Diana| 28| Bordeaux|
+-------+---+---------+



In [8]:
# Trier par une colonne
df.orderBy("age").show()

+-------+---+---------+
|    nom|age|    ville|
+-------+---+---------+
|    Bob| 25|     Lyon|
|  Diana| 28| Bordeaux|
|  Alice| 30|    Paris|
|Charlie| 35|Marseille|
+-------+---+---------+



In [9]:
# Ajouter une nouvelle colonne
df_avec_bonus = df.withColumn("age_dans_10_ans", df.age + 10)
df_avec_bonus.show()

+-------+---+---------+---------------+
|    nom|age|    ville|age_dans_10_ans|
+-------+---+---------+---------------+
|  Alice| 30|    Paris|             40|
|    Bob| 25|     Lyon|             35|
|Charlie| 35|Marseille|             45|
|  Diana| 28| Bordeaux|             38|
+-------+---+---------+---------------+



## 7. Difference importante : Lazy Evaluation

Spark utilise l'**evaluation paresseuse** (lazy evaluation) :
- Les transformations (select, filter, etc.) ne sont PAS executees immediatement
- Elles sont executees seulement quand on demande un resultat (show, count, collect)

```
TRANSFORMATIONS (lazy)          ACTIONS (declenchent l'execution)
--------------------            ---------------------------------
select()                        show()
filter()                        count()
withColumn()                    collect()
orderBy()                       write()
groupBy()                       first()
```

### Pourquoi ?
Spark peut optimiser l'ensemble des transformations avant de les executer.

---

## Exercice

**Objectif** : Creer et manipuler un DataFrame Spark

**Consigne** :
1. Creez un DataFrame avec des donnees de produits (nom, prix, quantite)
2. Affichez le schema
3. Filtrez les produits dont le prix est superieur a 20
4. Ajoutez une colonne "valeur_stock" = prix * quantite
5. Triez par valeur_stock decroissante

A vous de jouer :

In [10]:
# Donnees de produits
produits = [
    ("Laptop", 999.99, 10),
    ("Souris", 29.99, 100),
    ("Clavier", 79.99, 50),
    ("Ecran", 299.99, 25),
    ("Cable USB", 9.99, 200)
]

colonnes = ["nom", "prix", "quantite"]

# TODO: Creez le DataFrame
df_produits = spark.createDataFrame(produits, colonnes)
df_produits.show()

# TODO: Affichez le schema
df_produits.printSchema()

+---------+------+--------+
|      nom|  prix|quantite|
+---------+------+--------+
|   Laptop|999.99|      10|
|   Souris| 29.99|     100|
|  Clavier| 79.99|      50|
|    Ecran|299.99|      25|
|Cable USB|  9.99|     200|
+---------+------+--------+

root
 |-- nom: string (nullable = true)
 |-- prix: double (nullable = true)
 |-- quantite: long (nullable = true)



In [11]:
# TODO: Filtrez les produits avec prix > 20
df_produits.filter(df_produits.prix > 20).show()

+-------+------+--------+
|    nom|  prix|quantite|
+-------+------+--------+
| Laptop|999.99|      10|
| Souris| 29.99|     100|
|Clavier| 79.99|      50|
|  Ecran|299.99|      25|
+-------+------+--------+



In [12]:
# TODO: Ajoutez la colonne valeur_stock et triez
# Ajout de la colonne valeur_stock
new_df_produits = df_produits.withColumn(
    "valeur_stock",
    df_produits.prix * df_produits.quantite
)

# Tri décroissant
new_df_produits_sorted = new_df_produits.orderBy("valeur_stock", ascending=False)

# Affichage
new_df_produits_sorted.show()


+---------+------+--------+------------------+
|      nom|  prix|quantite|      valeur_stock|
+---------+------+--------+------------------+
|   Laptop|999.99|      10|            9999.9|
|    Ecran|299.99|      25|           7499.75|
|  Clavier| 79.99|      50|3999.4999999999995|
|   Souris| 29.99|     100|            2999.0|
|Cable USB|  9.99|     200|            1998.0|
+---------+------+--------+------------------+



---

## Resume

Dans ce notebook, vous avez appris :
- **Apache Spark** est un moteur de traitement distribue
- La **SparkSession** est le point d'entree pour utiliser Spark
- Un **DataFrame Spark** est similaire a Pandas mais distribue
- Les operations de base : select, filter, orderBy, withColumn
- La **lazy evaluation** : les transformations ne s'executent qu'a l'action

### Prochaine etape
Dans le prochain notebook, nous apprendrons a lire et ecrire des fichiers avec Spark.